In [ ]:
from sklearn.datasets import fetch_openml
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
mnist=fetch_openml('mnist_784',version=1)
mnist.keys()

In [ ]:
X,y=mnist["data"],mnist["target"]
X.shape


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

some_digit = X.iloc[0]
some_digit_array = np.array(some_digit)  # Convert Series to NumPy array
some_digit_image = some_digit_array.reshape(28, 28)

plt.imshow(some_digit_image, cmap="binary")
plt.axis("off")  # Turn off axis
plt.show()


In [ ]:
X_train = X[:60000]	#training set 60k
X_test = X[60000:]	#testing set 10k
y_train = y[:60000]	#training set label 
y_test = y[60000:]	#testing set label

In [ ]:
y_train_5 = (y_train == '5')
y_test_5 = (y_test == '5')

In [ ]:
from sklearn.linear_model import SGDClassifier

In [ ]:
sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit (X_train,y_train_5)

In [ ]:
sgd_clf.predict([some_digit])

In [ ]:
from sklearn.model_selection import cross_val_score

cross_val_score(sgd_clf,X_train,y_train_5, cv=3, scoring="accuracy")

# **Cross Validation (Self Implementation)**

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone

skfolds = StratifiedKFold(n_splits=3)

for train_index, test_index in skfolds.split(X_train, y_train_5):
    clone_clf = clone(sgd_clf)
    X_train_folds = X_train.iloc[train_index]
    y_train_folds = y_train_5.iloc[train_index]
    X_test_fold = X_train.iloc[test_index]
    y_test_fold = y_train_5.iloc[test_index]
    
    
    clone_clf.fit(X_train_folds,y_train_folds)
    y_pred = clone_clf.predict(X_test_fold)
    n_correct = sum(y_pred == y_test_fold)
    print (n_correct / len(y_pred))

In [ ]:
from sklearn.dummy import DummyClassifier
dummy_clf = DummyClassifier()
dummy_clf.fit(X_train, y_train_5)
print (any(dummy_clf.predict(X_train)))

In [ ]:
cross_val_score(dummy_clf, X_train, y_train_5 ,cv=3,scoring="accuracy")

In [ ]:
from sklearn.model_selection import cross_val_predict

cross_val_predict(sgd_clf,X_train,y_train_5, cv=3)

In [ ]:
from sklearn.model_selection import cross_val_predict

y_train_pred = cross_val_predict(sgd_clf,X_train,y_train_5, cv=3)

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_train_5,y_train_pred)
cm

In [ ]:
y_train_perfect_predection = y_train_5
confusion_matrix(y_train_5,y_train_perfect_predection)

In [ ]:
from sklearn.metrics import precision_score , recall_score

precision_score(y_train_5,y_train_pred)

In [ ]:
cm[1,1]/(cm[0,1]+cm[1,1])

In [ ]:
recall_score (y_train_5,y_train_pred)

In [ ]:
cm[1,1] / (cm[1,0] + cm[1,1])

In [ ]:
from sklearn.metrics import precision_recall_curve

precisions,recalls,thresholds = precision_recall_curve(y_train_5,y_train_pred)
plt.show()

# **new notbook**

# **Import necessary libraries**

In [ ]:
import pandas as pd
import numpy as np
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
plant_data = pd.read_csv('plant_disease_data.csv')

# Display basic information about the dataset
print(plant_data.info())
print(plant_data.describe())

# Display the number of classes and distribution of classes
print("Number of classes:", plant_data['label'].nunique())
print("Distribution of classes:\n", plant_data['label'].value_counts())

# Function to preprocess the images
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (128, 128))
    image = image / 255.0  # Normalize the image
    return image

# Apply preprocessing to the dataset
plant_data['image'] = plant_data['image_path'].apply(preprocess_image)

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
plant_data = pd.read_csv('plant_disease_data.csv')

# Display basic information about the dataset
print(plant_data.info())
print(plant_data.describe())

# Display the number of classes and distribution of classes
print("Number of classes:", plant_data['label'].nunique())
print("Distribution of classes:\n", plant_data['label'].value_counts())

# Function to preprocess the images
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (128, 128))
    image = image / 255.0  # Normalize the image
    return image

# Apply preprocessing to the dataset
plant_data['image'] = plant_data['image_path'].apply(preprocess_image)

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
# Extract images and labels
X = np.array(plant_data['image'].tolist())
y = plant_data['label'].values

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

print("Training set size:", len(X_train))
print("Test set size:", len(X_test))


In [ ]:
# Design a CNN architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

model.summary()


In [ ]:
# Compile and train the model
model.compile(optimizer=Adam(learning_rate=0.001), 
              loss=SparseCategoricalCrossentropy(), 
              metrics=['accuracy'])

history = model.fit(datagen.flow(X_train, y_train, batch_size=32), 
                    validation_data=(X_test, y_test), 
                    epochs=20)


In [ ]:
# Predictions on test set
y_pred = np.argmax(model.predict(X_test), axis=1)

# Accuracy and F1-score
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

print("Accuracy:", accuracy)
print("F1-Score:", f1)


In [ ]:
# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Sample predictions
sample_images = X_test[:5]
sample_labels = y_test[:5]
sample_preds = y_pred[:5]

for i in range(len(sample_images)):
    plt.imshow(sample_images[i])
    plt.title(f"Actual: {label_encoder.inverse_transform([sample_labels[i]])[0]}, Predicted: {label_encoder.inverse_transform([sample_preds[i]])[0]}")
    plt.show()


In [ ]:
# Transfer Learning Consideration
Transfer learning involves leveraging a pre-trained model (trained on a large dataset) to improve performance on a target task with limited data. For plant disease classification, a model like ResNet50 pre-trained on ImageNet can be fine-tuned to achieve better performance. It has learned general features that can be beneficial for identifying patterns in plant diseases.


In [ ]:
# Example with different learning rates
learning_rates = [0.001, 0.0001, 0.00001]
histories = []

for lr in learning_rates:
    model.compile(optimizer=Adam(learning_rate=lr), 
                  loss=SparseCategoricalCrossentropy(), 
                  metrics=['accuracy'])
    history = model.fit(datagen.flow(X_train, y_train, batch_size=32), 
                        validation_data=(X_test, y_test), 
                        epochs=10)
    histories.append(history)

# Plot training history for different learning rates
for i, history in enumerate(histories):
    plt.plot(history.history['accuracy'], label=f'LR: {learning_rates[i]}')
    
plt.title('Effect of Learning Rate on Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


# **New notbook**

# **Import necessary libraries**

In [1]:
import pandas as pd
import numpy as np
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

2024-06-26 05:47:01.744920: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-26 05:47:01.745024: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-26 05:47:01.862196: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Load the dataset and display basic statistics:



In [ ]:
# Load the dataset
plant_data = pd.read_csv('plant_disease_data.csv')

# Display basic information about the dataset
print(plant_data.info())
print(plant_data.describe())

# Display the number of classes and distribution of classes
print("Number of classes:", plant_data['label'].nunique())
print("Distribution of classes:\n", plant_data['label'].value_counts())

Preprocess the images:


In [ ]:
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Function to preprocess the images
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (128, 128))
    image = image / 255.0  # Normalize the image
    return image

# Apply preprocessing to the dataset
plant_data['image'] = plant_data['image_path'].apply(preprocess_image)

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

Split the data into training and testing sets:



In [ ]:
from sklearn.model_selection import train_test_split

# Assuming 'image' column has preprocessed images and 'label' column has labels
X = np.array(plant_data['image'].tolist())
y = plant_data['label'].values

# Encode labels
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

print("Training set size:", len(X_train))
print("Test set size:", len(X_test))

Design a CNN architecture:



In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

model.summary()


Compile and train the model:



In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

model.compile(optimizer=Adam(learning_rate=0.001), 
              loss=SparseCategoricalCrossentropy(), 
              metrics=['accuracy'])

history = model.fit(datagen.flow(X_train, y_train, batch_size=32), 
                    validation_data=(X_test, y_test), 
                    epochs=20)


Evaluate the model:



In [ ]:
from sklearn.metrics import accuracy_score, f1_score

# Predictions on test set
y_pred = np.argmax(model.predict(X_test), axis=1)

# Accuracy and F1-score
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

print("Accuracy:", accuracy)
print("F1-Score:", f1)


Visualize the model's performance:



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Sample predictions
sample_images = X_test[:5]
sample_labels = y_test[:5]
sample_preds = y_pred[:5]

for i in range(len(sample_images)):
    plt.imshow(sample_images[i])
    plt.title(f"Actual: {label_encoder.inverse_transform([sample_labels[i]])[0]}, Predicted: {label_encoder.inverse_transform([sample_preds[i]])[0]}")
    plt.show()


Explore the impact of adjusting the learning rate:



In [ ]:
# Example with different learning rates
learning_rates = [0.001, 0.0001, 0.00001]
histories = []

for lr in learning_rates:
    model.compile(optimizer=Adam(learning_rate=lr), 
                  loss=SparseCategoricalCrossentropy(), 
                  metrics=['accuracy'])
    history = model.fit(datagen.flow(X_train, y_train, batch_size=32), 
                        validation_data=(X_test, y_test), 
                        epochs=10)
    histories.append(history)

# Plot training history for different learning rates
for i, history in enumerate(histories):
    plt.plot(history.history['accuracy'], label=f'LR: {learning_rates[i]}')
    
plt.title('Effect of Learning Rate on Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()
